In [248]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

# General imports
import numpy as np

from qiskit.quantum_info import SparsePauliOp

# custom module
from susy_qm import calculate_Hamiltonian

from scipy.optimize import minimize, differential_evolution

import json
import os

In [204]:
potential = 'DW'
cutoff = 16
shots = None

In [205]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])
min_eigenvalue = min(eigenvalues.real)

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [206]:
min_eigenvalue

np.float64(0.8915993623272539)

In [207]:
cnot_pool = []
cz_pool = []

for control in range(num_qubits):
        for target in range(num_qubits):
            if control != target:
                cnot_pool.append(qml.CNOT(wires=[control, target]))
                cz_pool.append(qml.CZ(wires=[control, target]))

In [208]:
rot_pool = [qml.Rot(0.0, 0.0, 0.0, wires=x) for x in range(num_qubits)]
operator_pool = rot_pool +  cz_pool #+ cnot_pool

In [209]:
operator_pool

[Rot(0.0, 0.0, 0.0, wires=[0]),
 Rot(0.0, 0.0, 0.0, wires=[1]),
 Rot(0.0, 0.0, 0.0, wires=[2]),
 Rot(0.0, 0.0, 0.0, wires=[3]),
 Rot(0.0, 0.0, 0.0, wires=[4]),
 CZ(wires=[0, 1]),
 CZ(wires=[0, 2]),
 CZ(wires=[0, 3]),
 CZ(wires=[0, 4]),
 CZ(wires=[1, 0]),
 CZ(wires=[1, 2]),
 CZ(wires=[1, 3]),
 CZ(wires=[1, 4]),
 CZ(wires=[2, 0]),
 CZ(wires=[2, 1]),
 CZ(wires=[2, 3]),
 CZ(wires=[2, 4]),
 CZ(wires=[3, 0]),
 CZ(wires=[3, 1]),
 CZ(wires=[3, 2]),
 CZ(wires=[3, 4]),
 CZ(wires=[4, 0]),
 CZ(wires=[4, 1]),
 CZ(wires=[4, 2]),
 CZ(wires=[4, 3])]

In [210]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)

@qml.qnode(dev)
def circuit(params, trial_op, op_list):

    if len(op_list) > 0:
        for o, p, w, _ in op_list:
            if (o == qml.CNOT) | (o == qml.CZ):
                o(wires=w)
            else:
                o(*p, wires=w)

    op = type(trial_op)

    if (type(trial_op) == qml.CNOT) | (type(trial_op) == qml.CZ):
        op(wires=trial_op.wires)
    else:
        op(*params, wires=trial_op.wires)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [211]:
def cost_function(params, trial_op, op_list):
    
    params = pnp.tensor(params, requires_grad=True)
    energy = circuit(params, trial_op, op_list)

    return energy

In [212]:
num_steps = 20

x0 = np.random.uniform(0, 2 * np.pi, size=3)

op_list = []

for i in range(num_steps):

    print(f"step: {i}")

    energies = []
    e_params = []

    for trial_op in operator_pool:

        res = minimize(
                cost_function,
                x0,
                args=(trial_op, op_list),
                method= "COBYLA",
                options= {'maxiter':1000, 'tol': 1e-6}
            )
        
        energies.append(res.fun)
        e_params.append(res.x)

    min_arg = np.argmin(energies)
    min_energy = energies[min_arg]
    print(f"Min energy: {min_energy}")

    min_op = type(operator_pool[min_arg])
    min_wires = operator_pool[min_arg].wires
    min_params = e_params[min_arg]

    if (i != 0):
        if np.abs(min_energy - op_list[i-1][3]) < 1e-4:
            print("Converged")
            break

    op_list.append((min_op, min_params, min_wires, min_energy))

    

step: 0
Min energy: 1.0939110867557253
step: 1
Min energy: 1.0928892179864385
step: 2
Min energy: 1.0719155209353128
step: 3
Min energy: 1.0620654159866363
step: 4
Min energy: 1.0595874932667053
step: 5
Min energy: 1.0582536622216903
step: 6
Min energy: 1.0576523529005626
step: 7
Min energy: 1.05445420733646
step: 8
Min energy: 1.0492947451313732
step: 9
Min energy: 1.0482949221840192
step: 10
Min energy: 1.047974104996463
step: 11
Min energy: 1.0478439574073861
step: 12
Min energy: 1.04776009224311
Converged


In [226]:
last_operator = {}
reduced_op_list = []
num_params = 0

for o, p, w, _ in op_list:

    if (o == qml.CNOT) | (o == qml.CZ):
        last_operator[w[0]] = o
        last_operator[w[1]] = o
        reduced_op_list.append((o,w))

    elif w[0] in last_operator.keys():
        if last_operator[w[0]] == o:
            print("Operator already added previously... skipping")
            continue
        else:
            last_operator[w[0]] = o
            reduced_op_list.append((o,w))
            num_params = num_params + 3
    else:
        last_operator[w[0]] = o
        reduced_op_list.append((o,w))
        num_params = num_params + 3

Operator already added previously... skipping
Operator already added previously... skipping


In [250]:
data = {"potential": potential,
        "cutoff": cutoff,
        "optimizer": "COBYLA",
        "num steps": num_steps,
        "op_list": [str(x) for x in op_list],
        "reduced_op_list": [str(x) for x in reduced_op_list],
        "num_params": num_params}

In [253]:
# Save the variable to a JSON file
base_path = r"C:\Users\johnkerf\Desktop\Quantum-Computing\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files\\"
os.makedirs(base_path, exist_ok=True)
path = base_path + "{}_{}.json".format(potential, cutoff)
with open(path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [245]:
params = np.random.uniform(0, 2 * np.pi, size=num_params)

dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def construct_ansatz(reduced_op_list, params):

    params_index = 0

    for i, (o, w) in enumerate(reduced_op_list):
        if (o == qml.CNOT) | (o == qml.CZ):
            o(wires=w)
        else:
            num_gate_params = o.num_params
            o(*params[params_index:(params_index + num_gate_params)], wires=w)
            params_index = params_index + num_gate_params

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [246]:
print(qml.draw(construct_ansatz)(reduced_op_list, params))

0: ──────────────────────────────────────────────────────────────────────────────────────────────┤
1: ──────────────────────────────────────────────────────────────────────────────────────────────┤
2: ──Rot(3.64,0.08,4.39)─────────────────────────────────────────────────────────────────────────┤
3: ──Rot(2.37,5.91,5.11)─╭●──Rot(5.09,6.27,3.93)─╭●──Rot(3.90,3.91,1.46)─╭●──────────────────────┤
4: ──Rot(2.00,3.71,5.48)─╰Z──Rot(3.02,0.70,3.97)─╰Z──────────────────────╰Z──Rot(4.51,1.14,1.37)─┤

  ╭<𝓗(M0)>
  ├<𝓗(M0)>
  ├<𝓗(M0)>
  ├<𝓗(M0)>
  ╰<𝓗(M0)>

M0 = 
[[  2.375     +0.j   2.47487373+0.j   1.76776695+0.j ...   0.        +0.j
    0.        +0.j   0.        +0.j]
 [  2.47487373+0.j   5.875     +0.j   5.        +0.j ...   0.        +0.j
    0.        +0.j   0.        +0.j]
 [  1.76776695+0.j   5.        +0.j  10.875     +0.j ...   0.        +0.j
    0.        +0.j   0.        +0.j]
 ...
 [  0.        +0.j   0.        +0.j   0.        +0.j ... 163.875     +0.j
   55.56077753+0.j  83.3254162

In [254]:
qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=num_qubits)

(1, 5, 3)